# 1 Import Package

In [ ]:
import numpy as np
import pandas as pd

import tqdm
import os
import math

import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset, random_split
from torch.utils.tensorboard import SummaryWriter

# 2 Some helper functions

In [ ]:
# 2 Define some useful functions


import test


def same_seed(seed):
    """Fixed random seed to make results reproducible"""
    
    # 设置cudnn后端为确定性模式, 这可以保证每次运行网络的时候，相同输入的输出是确定的
    # 但是这样会影响性能，因为cudnn的确定性模式会禁用一些优化算法
    torch.backends.cudnn.deterministic = True
    
    # 关闭cudnn的benchmark模式, benchmark模式会自动寻找最适合当前配置的高效算法
    # 这样会导致每次运行的时候，相同输入的输出不一定是相同的
    torch.backends.cudnn.benchmark = False
    
    # 设置numpy的随机种子
    np.random.seed(seed)
    
    # 设置Pytorch的全局随机种子, 会影响模型从初始化、随机丢弃层等操作的随机性
    torch.manual_seed(seed)
    
    # 检查是否有GPU可用, 如果有GPU的话，则设置cuda的随机种子
    # 这段代码是深度学习实验中的最佳实践之一，特别是在需要复现结果或进行对比实验时非常重要。
    # 需要注意的是，即使设置了这些种子，在多 GPU 环境下可能还需要额外的设置才能完全保证结果的可重复性, 如下面的代码所示
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        
        
def train_valid_split(dataset, val_size=0.2, batch_size=32, seed=2021):
    """Split a dataset into a training dataset and a validation dataset"""
    
    # 为保证每次运行的结果一致，设置随机种子
    same_seed(seed)
    
    # 计算验证集的数量
    val_length = int(len(dataset) * val_size)
    
    # 计算训练集的数量
    train_length = len(dataset) - val_length
    
    # 使用torch.utils.data.random_split进行划分
    train_dataset, valid_dataset = random_split(dataset, [train_length, val_length], 
                                                generator=torch.Generator().manual_seed(seed))
    
    return np.array(train_dataset), np.array(valid_dataset)


def predict(test_loader, model, device):
    """Predict the output of a test dataset using a trained model"""
    
    model.eval() # Set the model to evaluation mode
    test_pred = []
    for x in tqdm.tqdm(test_loader):
        with torch.no_grad():
            x = x.to(device)
            output = model(x)
            test_pred.extend(output.detach().cpu().reshape(-1, 1))
    print(test_pred)
    preds = torch.cat(test_pred, dim=0).numpy()
    return preds

# 3. Dataset and DataLoader

In [ ]:
class COVID19Dataset(Dataset):
    """COVID-19 X-ray image dataset
    x: Features
    y: Targets, if None, then return x only
    """
    
    def __init__(self, x, y = None):
        if y is None:
            self.y = y
        else:
            self.y = torch.tensor(y, dtype=torch.float32)
            
        self.x = torch.tensor(x, dtype=torch.float32)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        if self.y is None:
            return self.x[index]
        else:
            return self.x[index], self.y[index]
        

# 4. Define Model

In [ ]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )
        
    def forward(self, x):
        return self.model(x).squeeze(1)

# 5. Feature Selection

In [ ]:
def select_features(train_data, valid_data, test_data, select_all = True):
    y_train, y_valid = train_data[:, -1], valid_data[:, -1]
    x_train, x_valid, x_test = train_data[:, :-1], valid_data[:, :-1], test_data
    
    if select_all:
        feature_index = list(range(x_train.shape[1]))
    else:
        feature_index = [0, 1, 2, 3, 4] # can modify this line to select different features
        
    return x_train[:, feature_index], x_valid[:, feature_index], x_test[:, feature_index], y_train, y_valid



# 6. Traning and evaluation

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):
    # reduction = 'mean' means that the sum of the squared errors will be divided by the number of samples
    criterion = nn.MSELoss(reduction = 'mean') 
    
    # Define optimizer
    # optimizer = optim.Adam(model.parameters(), lr = config['learning_rate'])
    optimizer = torch.optim.SGD(model.parameters(), lr = config['learning_rate'], momentum=0.9)
    
    writer = SummaryWriter(log_dir = config['log_dir'])
    
    model_dir = config['model_dir']
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0
    
    for epoch in range(n_epochs):
        model.train() # Set the model to training mode
        
        loss_record = []
        
        # tqdm is a library that can display the progress bar
        # position = 0 means the progress bar will be displayed at the first line
        # leave = True means the progress bar will leave after finishing
        train_pbar = tqdm.tqdm(train_loader, position = 0, leave = True)
        
        for x, y in train_pbar:
            x, y = x.to(device), y.to(device) # Move your data to device.
            
            optimizer.zero_grad() # Set gradient to zero.
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()  # Compute gradient(backpropagation).
            optimizer.step() # Update parameters.
            
            step += 1
            
            # detach the loss to convert it to a scalar
            loss_record.append(loss.detach().item())
        
        mean_train_loss = sum(loss_record) / len(loss_record)
        writer.add_scalar('train_loss', mean_train_loss, epoch)
        
        # switch to evaluation mode
        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)
                loss_record.append(loss.item())
                
        mean_valid_loss = sum(loss_record) / len(loss_record)
        print(f"EPOCH: {epoch}/{n_epochs}, TRAIN LOSS: {mean_train_loss:.4f}, VALID LOSS: {mean_valid_loss:.4f}")
        writer.add_scalar('valid_loss', mean_valid_loss, epoch)
        
        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path'])
            print("Saving model with valid loss: {best_loss:.4f}")
            early_stop_count = 0
        else:
            early_stop_count += 1
            
        if early_stop_count > config['early_stop']:
            print("\nModel is not improving, so we halt the training session.")
            return 
        

# Configurations

`config` contains hyper-parameters for training and the path to save your model.

In [ ]:

from logging import config


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 3000,     # Number of epochs.
    'batch_size': 256,
    'learning_rate': 1e-5,
    'early_stop': 400,    # If model has not improved for this many consecutive epochs, stop training.
    "model_dir": "./models",  # Your model will be saved here.
    'save_path': './models/model.ckpt',  # Your best model save path
    "log_dir": "./logs"  # Your tensorboard log path
}

# Dataloader
Read data from files and set up training, validation, and testing sets. You do not need to modify this part.

In [ ]:
same_seed(config['seed'])

train_data = pd.read_csv('../Data/covid.train.csv').values
test_data = pd.read_csv('../Data/covid.test.csv').values

train_data, valid_data = train_valid_split(train_data, val_size=config['valid_ratio'], seed=config['seed'])

# print out the data size
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# select features
x_train, x_valid, x_test, y_train, y_valid = select_features(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset = COVID19Dataset(x_train, y_train)
valid_dataset = COVID19Dataset(x_valid, y_valid)
test_dataset = COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
# shuffle=True means the data will be shuffled.
# pin_memory=True means the data will be loaded to GPU memory
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)


# Start training

In [ ]:
model = My_Model(x_train.shape[1]).to(device) # Move model to device.
trainer(train_loader, valid_loader, model, config, device)

# Plot learning curves with `tensorboard` (optional)

`tensorboard` is a tool that allows you to visualize your training progress.

If this block does not display your learning curve, please wait for few minutes, and re-run this block. It might take some time to load your logging information.

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=./logs/

# Testing
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    
    preds_df = pd.DataFrame(preds, columns=['tested_positive'])
    preds_df.index.name = 'id'
    preds_df.to_csv(file)
    
model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')